## groupby 函数的用法

In [5]:
import pandas as pd
import numpy as np

In [33]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                      'key2' : ['one', 'two', 'one', 'two', 'one'],
                       'data1' : np.random.randn(5),
                      'data2' : np.random.randn(5)})

In [7]:
df

,data1,data2,key1,key2
0,-0.971774,-0.826209,a,one
1,0.218501,0.148096,a,two
2,1.078806,1.936013,b,one
3,1.430297,0.028241,b,two
4,0.939290,-0.763747,a,one


假设你想要按key1进行分组，并计算data1列的平均值。实现该功能的方式有很多，而我们这里要用的是：访问data1，并根据key1调用groupby：

In [8]:
grouped = df['data1'].groupby(df['key1'])
print(grouped)

变量grouped是一个GroupBy对象。**它实际上还没有进行任何计算**，只是含有一些有关分组键df['key1']的中间数据而已。换句话说，该对象已经有了接下来对各分组执行运算所需的一切信息。例如，我们可以调用GroupBy的mean方法来计算分组平均值：

In [9]:
grouped.mean()

key1
a    0.062005
b    1.254552
Name: data1, dtype: float64

这里最重要的是，数据（Series）**根据分组键进行了聚合，产生了一个新的Series**，其索引为key1列中的唯一值。之所以结果中索引的名称为key1，是因为原始DataFrame的列df['key1']就叫这个名字。

如果我们一次传入多个数组的列表，就会得到不同的结果：

In [10]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.016242
      two     0.218501
b     one     1.078806
      two     1.430297
Name: data1, dtype: float64

我通过两个键对数据进行了分组，得到的Series具有一个层次化索引（由唯一的键对组成）：

In [14]:
means.unstack(level = -1)

key2,one,two
key1,,
a,-0.016242,0.218501
b,1.078806,1.430297


在这个例子中，分组键均为Series。实际上，分组键可以是任何长度适当的数组：

In [15]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

你还可以将列名（可以是字符串、数字或其他Python对象）用作分组键

In [16]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.062005,-0.480620
b,1.254552,0.982127


# 对分组进行迭代

GroupBy对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）。

In [17]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -0.971774 -0.826209    a  one
1  0.218501  0.148096    a  two
4  0.939290 -0.763747    a  one
b
      data1     data2 key1 key2
2  1.078806  1.936013    b  one
3  1.430297  0.028241    b  two


对于多重键的情况，元组的第一个元素将会是由键值组成的元组：

In [18]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
   ....:     print((k1, k2))
   ....:     print(group)

('a', 'one')
      data1     data2 key1 key2
0 -0.971774 -0.826209    a  one
4  0.939290 -0.763747    a  one
('a', 'two')
      data1     data2 key1 key2
1  0.218501  0.148096    a  two
('b', 'one')
      data1     data2 key1 key2
2  1.078806  1.936013    b  one
('b', 'two')
      data1     data2 key1 key2
3  1.430297  0.028241    b  two


可以对这些数据片段做任何操作。有一个你可能会觉得有用的运算：将这些数据片段做成一个字典：

In [19]:
 pieces = dict(list(df.groupby('key1')))

In [20]:
pieces['b']

,data1,data2,key1,key2
2,1.078806,1.936013,b,one
3,1.430297,0.028241,b,two


# 选取一列或列的子集

对于由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。

In [21]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

也可以表示为

In [22]:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

尤其对于大数据集，很可能只需要对部分列进行聚合。例如，在前面那个数据集中，如果只需计算data2列的平均值并以DataFrame形式得到结果，可以这样写：

In [23]:
 df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.794978
     two   0.148096
b    one   1.936013
     two   0.028241

## 通过字典或Series进行分组
除数组以外，分组信息还可以其他形式存在。

In [24]:
people = pd.DataFrame(np.random.randn(5, 5),
   ....:                       columns=['a', 'b', 'c', 'd', 'e'],
   ....:                       index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])


In [25]:
people

,a,b,c,d,e
Joe,0.717591,-1.374042,1.233309,0.836459,-1.445321
Steve,1.113686,-0.220040,1.757240,-0.851394,0.760859
Wes,-0.687228,-0.818290,-0.350860,0.509322,0.259932
Jim,-0.797768,1.140872,-0.530692,0.229211,1.623305
Travis,-0.278991,-1.303252,1.488004,-0.142046,0.182245


In [26]:
people.iloc[2:3, [1, 2]] = np.nan

In [27]:
people

,a,b,c,d,e
Joe,0.717591,-1.374042,1.233309,0.836459,-1.445321
Steve,1.113686,-0.220040,1.757240,-0.851394,0.760859
Wes,-0.687228,NaN,NaN,0.509322,0.259932
Jim,-0.797768,1.140872,-0.530692,0.229211,1.623305
Travis,-0.278991,-1.303252,1.488004,-0.142046,0.182245


现在，假设已知列的分组关系，并希望根据分组计算列的和：

In [28]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
   ....:            'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [29]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,2.069767,-2.101773
Steve,0.905846,1.654505
Wes,0.509322,-0.427296
Jim,-0.301482,1.966408
Travis,1.345958,-1.399998


Series也有同样的功能，它可以被看做一个固定大小的映射：

In [30]:
map_series = pd.Series(mapping)

In [31]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

如果要使用你自己的聚合函数，只需将其传入aggregate或agg方法即可

In [32]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [34]:
df


,data1,data2,key1,key2
0,1.625696,0.705196,a,one
1,-0.377549,0.373419,a,two
2,1.533781,-0.143352,b,one
3,0.758572,-0.738757,b,two
4,1.694688,0.803694,a,one


In [35]:
grouped = df.groupby('key1')

In [36]:
def peak_to_peak(arr):
   ....:     return arr.max() - arr.min()